In [1]:
import pulp

https://www.lemonde.fr/sciences/article/2025/10/11/le-code-l-enigme-maths-du-monde-n-72_6645741_1650684.html

In [95]:
# create problem
prob = pulp.LpProblem("lines", pulp.LpMaximize)

#m = pulp.LpVariable("m",cat=pulp.LpInteger)
#n = pulp.LpVariable("n",cat=pulp.LpInteger)
# x[i][j]=1 <=> digit j = i for m
x = pulp.LpVariable.dicts("x",([i for i in range(10)],[j for j in range(10)]),cat=pulp.LpBinary)
# y[i][j]=1 <=> digit j = i for n
y = pulp.LpVariable.dicts("y",([i for i in range(10)],[j for j in range(10)]),cat=pulp.LpBinary)
# z[j]=1 <=> digit j is >0 for m or n
z = pulp.LpVariable.dicts("z",[j for j in range(10)],cat=pulp.LpBinary)

# objective
#prob += m+n
prob+= pulp.lpSum(i*x[i][j]*10**j for i in range(10) for j in range(10))+pulp.lpSum(i*y[i][j]*10**j for i in range(10) for j in range(10))

# m in base 10
#prob+= m==pulp.lpSum(i*x[i][j]*10**j for i in range(10) for j in range(10))
for j in range(10):
    prob+= pulp.lpSum(x[i][j] for i in range(10))==1

# n in base 10
#prob+= n==pulp.lpSum(i*y[i][j]*10**j for i in range(10) for j in range(10))
for j in range(10):
    prob+= pulp.lpSum(y[i][j] for i in range(10))==1

# numbers 1-9 can only appear once
# 0 can appear multiple times ! but only at the left of the the non zero digit
for i in range(1,10):
    prob+= pulp.lpSum(x[i][j]+y[i][j] for j in range(10))<=1, "unique_%s"%i

# digitwise sum cannot be under 10
for j in range(10):
    if j==0:
        prob+= pulp.lpSum(i*x[i][j]+i*y[i][j] for i in range(10))>=10
    else:
        prob+= pulp.lpSum(i*x[i][j]+i*y[i][j] for i in range(10))>=9*z[j]
    # z[j] is active is i is > 0
    prob+= pulp.lpSum(i*x[i][j] for i in range(10)) <= 9*z[j]
    prob+= pulp.lpSum(i*y[i][j] for i in range(10)) <= 9*z[j]

# 0 can appear multiple times ! but only at the left of the the non zero digit
for j in range(10-1):
    prob+= 1-z[j] <= 1-z[j+1]

In [96]:
#prob

In [97]:
prob.solve()

1

In [98]:
#print(pulp.value(m),"+",pulp.value(n),"=",pulp.value(prob.objective))

In [99]:
#pulp.value(m)+pulp.value(n)

In [100]:
#for j in range(10):
#    for i in range(10):
#        val=pulp.value(x[i][j])
#        if val>0.9:
#            print("digit",j,10**j,"->",i)

In [102]:
print("m = ",pulp.lpSum(i*pulp.value(x[i][j])*10**j for i in range(10) for j in range(10)))
print("n = ",pulp.lpSum(i*pulp.value(y[i][j])*10**j for i in range(10) for j in range(10)))
print("m+n =",pulp.lpSum(i*pulp.value(x[i][j])*10**j for i in range(10) for j in range(10))+pulp.lpSum(i*pulp.value(y[i][j])*10**j for i in range(10) for j in range(10)))

m =  7648.0
n =  9352.0
m+n = 17000.0
